In [1]:
import turicreate as tc

In [2]:
import json

# Name your model

In [3]:
modelName = "MyModel"

# Build train JSON SFrame

In [4]:
with open('annotations.json') as j:
    annotations = json.load(j)

In [5]:
annotationData = tc.SFrame(annotations)

In [6]:
data = tc.load_images('images/')

In [7]:
data = data.join(annotationData)

In [8]:
trainData, testData = data.random_split(0.8)

# Check ground truth

In [9]:
trainData['image_with_ground_truth'] = \
    tc.object_detector.util.draw_bounding_boxes(trainData['image'], trainData['annotations'])

In [10]:
trainData.explore()

Materializing SFrame

# Train the model

In [11]:
model = tc.object_detector.create(trainData, feature="image", annotations="annotations", max_iterations=30)

Setting 'batch_size' to 32
Using CPU to create model
NOTE: If available, an AMD GPU can be leveraged on macOS 10.14+ for faster model creation
+--------------+--------------+--------------+
| Iteration    | Loss         | Elapsed Time |
+--------------+--------------+--------------+
| 1            | 4.544        | 10.3         |
| 3            | 4.558        | 29.6         |
| 5            | 4.566        | 48.6         |
| 7            | 4.557        | 67.9         |
| 9            | 4.558        | 87.4         |
| 11           | 4.478        | 106.9        |
| 13           | 4.485        | 126.2        |
| 15           | 4.459        | 146.0        |
| 17           | 4.378        | 165.6        |
| 19           | 4.315        | 184.9        |
| 21           | 4.288        | 204.5        |
| 23           | 4.246        | 224.1        |
| 25           | 4.206        | 244.0        |
| 27           | 4.173        | 263.3        |
| 29           | 4.119        | 282.7        |
| 30       

In [12]:
model.save(modelName + '.model')

# Predictions

In [13]:
predictions = model.predict(testData, confidence_threshold=0.0, verbose=True)

Predicting  1/14
Predicting 14/14


In [14]:
predictions.explore()

Materializing SFrame

In [15]:
metrics = model.evaluate(testData)

Predicting  1/14
Predicting 14/14


In [16]:
print('mAP: {:.1%}'.format(metrics['mean_average_precision_50']))

mAP: 30.6%


In [17]:
metrics

{'average_precision_50': {'Feder': 0.30599078264015506},
 'mean_average_precision_50': 0.30599078264015506}

# Export model

In [18]:
model.export_coreml(modelName + '.mlmodel')